In [ ]:
import pandas as pd
import numpy as np
import json
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import string
from nltk.tokenize import word_tokenize
import nltk
import scipy.sparse as sp
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
!jupyter nbextension enable --py widgetsnbextension
df = pd.read_csv("../data/raw/20_newsgroups_test.csv")
len(df)

7532

In [56]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Load stopwords
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase
    text = str(text).lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numeric characters
    #text = re.sub(r'\d+', '', text)
    # Tokenize
    tokens = text.split()
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Return the preprocessed text as a single string
    return ' '.join(tokens)

def clean_text(text):
    """
    Basic text cleaning function.
    """
    if not isinstance(text, str):
        return text
    text = text.lower()  # Lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    return text.strip()

In [7]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have a DataFrame called df with columns 'text' and 'label'
# Example:
# df = pd.DataFrame({
#     'text': ['This is text one', 'This is text two', 'Another text', ...],
#     'label': [0, 1, 0, ...]
# })
# Split the data into training and testing sets
df['text'] = df['text'].apply(preprocess_text)
X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
df.loc[df[df['text'] == ''].index,"text"] = None
df.dropna(subset=["text"],inplace=True)


In [57]:
import pandas as pd
import numpy as np

text_column = "text"

df3 = pd.read_csv("../data/raw/20_newsgroups_test.csv")

df3[text_column] = df3[text_column].apply(clean_text)

print("Number of rows before removing empty texts:", len(df3))
df3.loc[df3[df3['text'] == ''].index,"text"] = None
df3.dropna(subset=["text"],inplace=True)
print("Number of rows after removing empty texts:", len(df3))

df3.loc[df3[df3['text'].isnull()].index,"text"] 

Number of rows before removing empty texts: 7532
Number of rows after removing empty texts: 7309


Series([], Name: text, dtype: object)

In [ ]:
df3.to_csv("test.csv",index=False)

In [61]:
dfx = pd.read_csv("../data/processed/20_newsgroups_test.csv")
dfx.loc[dfx[dfx['text'].isnull()].index] 

,text,label,label_text


In [10]:
from gensim.models import FastText, KeyedVectors

# Load the model
model = FastText.load("../models/embeddings/fasttext_20_newsgroups.model")


# Example: Working with words and subwords
word_vector = model.wv["example"]  # Get vector for a word
oov_vector = model.wv["nonexistentword"]  # Get vector for an out-of-vocabulary word (using n-grams)

# Print results
print("Word vector:", word_vector)  
print("OOV vector:", oov_vector)

Word vector: [ 8.70180726e-02 -1.18468076e-01 -5.24383746e-02  1.48949057e-01
 -2.17622295e-01  1.29811153e-01  2.51080722e-01  2.36998439e-01
 -2.31754214e-01  2.40379721e-01  1.78046957e-01 -4.59710620e-02
 -9.35861003e-03 -6.86649159e-02  4.55453336e-01 -1.60753265e-01
  3.64714533e-01 -1.20403455e-03 -9.79096442e-02  4.14732486e-01
 -2.45706774e-02  1.72870412e-01 -1.58728249e-02  1.53341547e-01
  7.77342319e-02  5.11758327e-02  2.33883694e-01  2.74856865e-01
  8.66269618e-02  8.26513469e-02 -2.43477505e-02  1.25872657e-01
 -2.33803168e-01  2.26431489e-01  5.00639975e-02 -2.07496062e-01
 -1.38248667e-01 -1.18811183e-01  3.30808572e-02 -1.45987079e-01
 -2.15115160e-01  1.14879109e-01  3.87630775e-03  1.81055233e-01
  2.96759367e-01  3.54611069e-01 -4.70352709e-01  5.11219092e-02
 -6.66729659e-02  1.74889401e-01  1.04263350e-01  2.33744904e-01
  3.96120697e-01  1.64655838e-02  1.10240772e-01 -9.10459608e-02
  5.02471514e-02 -1.68930918e-01 -7.25578666e-02  7.50460550e-02
 -4.15480196

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [14]:
df2 = df.copy()
from tqdm import tqdm
df2["embeddings"] = None
for idx, row in tqdm(df2.iterrows(), total=len(df2)):
    
    try:
        df2.at[idx, "embeddings"] = model.wv.get_mean_vector(row["text"].split())
    except:
        print(idx,"row",row)

100%|██████████| 7466/7466 [00:07<00:00, 950.46it/s] 


In [ ]:
from gensim.models import FastText

# Load a pre-trained FastText model
fasttext_model = FastText.load_fasttext_format('path/to/your/fasttext/model.bin')

# Example usage: Get the vector for a word
word_vector = fasttext_model['example']
print(word_vector)

In [ ]:
# Train a classifier
# Here, we use RandomForestClassifier but you can choose any classifier (e.g., SVM, Logistic Regression, etc.)
doc2vec_model = build_model(X_train)
X_train_vectors, X_test_vectors = infer_vectors(doc2vec_model, X_train), infer_vectors(doc2vec_model, X_test)
clf = RandomForestClassifier(n_estimators=300,random_state=42,verbose=1,n_jobs=8)
clf.fit(X_train_vectors, y_train)

# Predict and evaluate the model on the test set
y_pred = clf.predict(X_test_vectors)

# Compute accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    4.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   20.8s
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:   34.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:    0.2s finished


Accuracy: 0.2721875
Classification Report:
              precision    recall  f1-score   support

           0       0.36      0.13      0.19       301
           1       0.30      0.12      0.17       336
           2       0.35      0.50      0.42       454
           3       0.44      0.28      0.35       334
           4       0.16      0.04      0.06       209
           5       0.31      0.16      0.21       281
           6       0.24      0.38      0.29       567
           7       0.21      0.18      0.19       447
           8       0.15      0.06      0.08       595
           9       0.39      0.27      0.32       408
          10       0.33      0.54      0.41       554
          11       0.39      0.12      0.18       156
          12       0.00      0.00      0.00        90
          13       0.27      0.02      0.03       220
          14       0.27      0.47      0.35       758
          15       0.16      0.23      0.19       690

    accuracy                         

In [48]:
# Initialize and train the XGBoost model
def train_model(X_train, X_test, y_train, y_test):
    xgb_model = XGBClassifier(use_label_encoder=False, 
                              eval_metric='mlogloss', 
                              objective='multi:softmax', 
                              num_class= 16,
                              learning_rate=0.1, 
                              #max_depth=5, 
                              n_estimators=20,
                              gemma=1
                              )

    # Train the XGBoost model with early stopping on the validation set
    xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=True)
    return xgb_model

from sklearn.metrics import classification_report

def evaluate(_X_test:np.array, _y_test:list, xgb_model):
    _y_pred = xgb_model.predict(_X_test)
    accuracy = accuracy_score(_y_test, _y_pred)
    print(f"Accuracy: {accuracy}")
    print("Classification Report:\n",classification_report(_y_test, _y_pred)) 
    return accuracy

# Extract epoch-wise metrics
def visualize_model(xgb_model):
    from matplotlib import pyplot as plt
    results = xgb_model.evals_result_  # Get the evaluation results during training

    # Print results keys
    print("Keys in evals_result_:", results.keys())
    # Accessing specific metrics
    validation_loss = results['validation_0']['mlogloss']  # Epoch-wise loss on validation set
    epochs = range(1, len(validation_loss) + 1)  # Epoch numbers
    # Plot the loss over epochs
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, validation_loss, label='Validation Loss', marker='o')
    plt.title('Validation Loss Across Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Log Loss')
    plt.legend()
    plt.grid()
    plt.show()

In [49]:
model = train_model(X_train_vectors, X_test_vectors, y_train, y_test)
evaluate(X_test_vectors, y_test, model)

/Users/ngumus/projects/Master-Thesis/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [00:33:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "gemma", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-mlogloss:2.71583
[1]	validation_0-mlogloss:2.66875
[2]	validation_0-mlogloss:2.62810
[3]	validation_0-mlogloss:2.59179
[4]	validation_0-mlogloss:2.56201
[5]	validation_0-mlogloss:2.53393
[6]	validation_0-mlogloss:2.50653
[7]	validation_0-mlogloss:2.48187
[8]	validation_0-mlogloss:2.45986
[9]	validation_0-mlogloss:2.43875
[10]	validation_0-mlogloss:2.41934
[11]	validation_0-mlogloss:2.40094
[12]	validation_0-mlogloss:2.38423
[13]	validation_0-mlogloss:2.36868
[14]	validation_0-mlogloss:2.35326
[15]	validation_0-mlogloss:2.33820
[16]	validation_0-mlogloss:2.32558
[17]	validation_0-mlogloss:2.31211
[18]	validation_0-mlogloss:2.29966
[19]	validation_0-mlogloss:2.28672
Accuracy: 0.28796875
Classification Report:
               precision    recall  f1-score   support

           0       0.37      0.26      0.31       301
           1       0.30      0.23      0.26       336
           2       0.43      0.46      0.44       454
           3       0.36      0.23      0.28     

0.28796875

In [30]:
y_train

7125     Religion&Philosophy&Ethics
15247        Mechanical_engineering
1675                     Psychology
9919               Natural_sciences
15458             Civil_engineering
                    ...            
10776        Engineering&Technology
14332                       Biology
5318             Business&Economics
5053     Religion&Philosophy&Ethics
9542             Business&Economics
Name: category, Length: 12800, dtype: object